# INE 

In [1]:
#leer csv
def leer_archivo_rango_periodo(ruta_archivo, periodo):
    
    #leer archivo CSV
    ine_original = pd.read_csv(ruta_archivo, encoding='latin1',sep="\t")
    
    #crear copia del DF original
    ine_copia= ine_original.copy()
    
    #FIltrar el DataFrame para incluir solo las filas que contienen el periodo deseado
    ine_filtrado= ine_copia[ine_copia['Periodo'].str.contains(periodo)]
    
    ine_filtrado = ine_filtrado.reset_index(drop=True)
    
    return ine_filtrado

In [2]:
#limpieza de datos
def procesar_ine_data(df):
    
    column_rename_dict = {'ï»¿Totales Territoriales': 'Totales_Territoriales',
                      'Comunidades y Ciudades AutÃ³nomas': 'Comunidades_Ciudades_Autonomas',
                      'Provincias': 'Provincias',
                      'Viajeros y pernoctaciones': 'Viajeros_Pernoctaciones',
                      'Residencia: Nivel 1': 'Residencia_Nivel_1',
                      'Residencia: Nivel 2': 'Residencia_Nivel_2',
                      'Periodo': 'Periodo',
                      'Total': 'Total'}
    
    # Renombro las columnas usando el diccionario
    df.rename(columns=column_rename_dict, inplace=True)
    
    #Reseteo el indice
    df.reset_index(drop=True, inplace=True)
    
    # Filtro filas para obtener el DataFrame de Viajeros
    viajeros_df = df[df['Viajeros_Pernoctaciones'] == 'Viajero']

    # Filtro filas para obtener el DataFrame de Pernoctaciones
    pernoctaciones_df = df[df['Viajeros_Pernoctaciones'] == 'Pernoctaciones']

    # Reseteo los índices para ambos DataFrames
    viajeros_df.reset_index(drop=True, inplace=True)
    pernoctaciones_df.reset_index(drop=True, inplace=True)
    
    return viajeros_df, pernoctaciones_df

In [3]:
def procesar_viajeros(df):
    
    #'pivot_table' para reestructurar los datos. 
    viajeros= df.pivot_table(index= 'Provincias', columns= ('Periodo','Residencia_Nivel_2'), values= 'Total', aggfunc= 'sum')
    
    #Estandarizo el índice para que también pueda relacionarlo con el resto de fuentes a estudiar
    correccion_provincias = {'01 Araba/Ãlava': 'ARABA/ALAVA', 
                       '02 Albacete': 'ALBACETE', 
                       '03 Alicante/Alacant': 'ALICANTE', 
                       '04 AlmerÃ­a': 'ALMERIA',
                       '05 Ãvila': 'AVILA',
                       '06 Badajoz': 'BADAJOZ',
                       '07 Balears, Illes': 'ILLES BALEARS',
                       '08 Barcelona': 'BARCELONA',
                       '09 Burgos': 'BURGOS',
                       '10 CÃ¡ceres': 'CACERES',
                       '11 CÃ¡diz': 'CADIZ',
                       '12 CastellÃ³n/CastellÃ³': 'CASTELLON',
                       '13 Ciudad Real': 'CIUDAD REAL',
                       '14 CÃ³rdoba': 'CORDOBA',
                       '15 CoruÃ±a, A': 'A CORUÑA',
                       '16 Cuenca': 'CUENCA',
                       '17 Girona': 'GIRONA',
                       '18 Granada': 'GRANADA',
                       '19 Guadalajara': 'GUADALAJARA',
                       '20 Gipuzkoa': 'GIPUZKOA',
                       '21 Huelva': 'HUELVA',
                       '22 Huesca': 'HUESCA',
                       '23 JaÃ©n': 'JAEN',
                       '24 LeÃ³n': 'LEON',
                       '25 Lleida': 'LLEIDA',
                       '26 Rioja, La': 'LA RIOJA',
                       '27 Lugo': 'LUGO',
                       '28 Madrid': 'MADRID',
                       '29 MÃ¡laga': 'MALAGA',
                       '30 Murcia': 'MURCIA',
                       '31 Navarra':'NAVARRA',
                       '32 Ourense': 'OURENSE',
                       '33 Asturias': 'ASTURIAS',
                       '34 Palencia': 'PALENCIA',
                       '35 Palmas, Las': 'LAS PALMAS',
                       '36 Pontevedra': 'PONTEVEDRA',
                       '37 Salamanca':'SALAMANCA' ,
                       '38 Santa Cruz de Tenerife': 'STA. CRUZ DE TENERIFE',
                       '39 Cantabria': 'CANTABRIA',
                       '40 Segovia': 'SEGOVIA',
                       '41 Sevilla': 'SEVILLA',
                       '42 Soria': 'SORIA',
                       '43 Tarragona': 'TARRAGONA',
                       '44 Teruel': 'TERUEL',
                       '45 Toledo': 'TOLEDO',
                       '46 Valencia/ValÃ¨ncia': 'VALENCIA',
                       '47 Valladolid': 'VALLADOLID',
                       '48 Bizkaia': 'BIZKAIA',
                       '49 Zamora': 'ZAMORA',
                       '50 Zaragoza': 'ZARAGOZA'}
    #Corrijo el índice. Recurriré 'map' para aplicar una función a cada elemento del índice de este DF.
    viajeros.index =viajeros.index.map(correccion_provincias)
    
    #Primero me aseguro que 'viajeros' tenga el mismo índice antes de ordenarlo
    viajeros.reset_index(inplace=True)
    viajeros = viajeros.set_index('Provincias')
    
    #Establezco el índice para ordenarlos por orden alfabético:
    viajeros = viajeros.sort_index()
    
    #Ajusto los nombre de las columnas del segundo nivel. Accedo también al segundo nivel que es donde se encuentra.
    viajeros.columns = viajeros.columns.set_levels(viajeros.columns.levels[1].str.replace('Residentes en EspaÃ±a', 'Residentes en Espana'), level=1)
    
    #A continuación junto el primer nivel y el segundo para poder exportarlo correctamente a la Base de Datos:
    viajeros.columns = [e[0]+'_'+e[1] for e in viajeros.columns]
    
    return viajeros

In [4]:
def procesar_pernoctaciones(df, sufijo):
    
    #'pivot_table' para reestructurar los datos. 
    pernoctaciones= df.pivot_table(index= 'Provincias', columns= ('Periodo','Residencia_Nivel_2'), values= 'Total', aggfunc= 'sum')
    
    #Estandarizo el índice para que también pueda relacionarlo con el resto de fuentes a estudiar
    correccion_provincias = {'01 Araba/Ãlava': 'ARABA/ALAVA', 
                       '02 Albacete': 'ALBACETE', 
                       '03 Alicante/Alacant': 'ALICANTE', 
                       '04 AlmerÃ­a': 'ALMERIA',
                       '05 Ãvila': 'AVILA',
                       '06 Badajoz': 'BADAJOZ',
                       '07 Balears, Illes': 'ILLES BALEARS',
                       '08 Barcelona': 'BARCELONA',
                       '09 Burgos': 'BURGOS',
                       '10 CÃ¡ceres': 'CACERES',
                       '11 CÃ¡diz': 'CADIZ',
                       '12 CastellÃ³n/CastellÃ³': 'CASTELLON',
                       '13 Ciudad Real': 'CIUDAD REAL',
                       '14 CÃ³rdoba': 'CORDOBA',
                       '15 CoruÃ±a, A': 'A CORUÑA',
                       '16 Cuenca': 'CUENCA',
                       '17 Girona': 'GIRONA',
                       '18 Granada': 'GRANADA',
                       '19 Guadalajara': 'GUADALAJARA',
                       '20 Gipuzkoa': 'GIPUZKOA',
                       '21 Huelva': 'HUELVA',
                       '22 Huesca': 'HUESCA',
                       '23 JaÃ©n': 'JAEN',
                       '24 LeÃ³n': 'LEON',
                       '25 Lleida': 'LLEIDA',
                       '26 Rioja, La': 'LA RIOJA',
                       '27 Lugo': 'LUGO',
                       '28 Madrid': 'MADRID',
                       '29 MÃ¡laga': 'MALAGA',
                       '30 Murcia': 'MURCIA',
                       '31 Navarra':'NAVARRA',
                       '32 Ourense': 'OURENSE',
                       '33 Asturias': 'ASTURIAS',
                       '34 Palencia': 'PALENCIA',
                       '35 Palmas, Las': 'LAS PALMAS',
                       '36 Pontevedra': 'PONTEVEDRA',
                       '37 Salamanca':'SALAMANCA' ,
                       '38 Santa Cruz de Tenerife': 'STA. CRUZ DE TENERIFE',
                       '39 Cantabria': 'CANTABRIA',
                       '40 Segovia': 'SEGOVIA',
                       '41 Sevilla': 'SEVILLA',
                       '42 Soria': 'SORIA',
                       '43 Tarragona': 'TARRAGONA',
                       '44 Teruel': 'TERUEL',
                       '45 Toledo': 'TOLEDO',
                       '46 Valencia/ValÃ¨ncia': 'VALENCIA',
                       '47 Valladolid': 'VALLADOLID',
                       '48 Bizkaia': 'BIZKAIA',
                       '49 Zamora': 'ZAMORA',
                       '50 Zaragoza': 'ZARAGOZA'}
    
    #Corrijo el índice. Recurriré 'map' para aplicar una función a cada elemento del índice de este DF.
    pernoctaciones.index =pernoctaciones.index.map(correccion_provincias)
    
    #Primero me aseguro que 'viajeros' tenga el mismo índice antes de ordenarlo
    pernoctaciones.reset_index(inplace=True)
    pernoctaciones = pernoctaciones.set_index('Provincias')
    
    #Establezco el índice para ordenarlos por orden alfabético:
    pernoctaciones = pernoctaciones.sort_index()
    
    #Ajusto los nombre de las columnas del segundo nivel. Accedo también al segundo nivel que es donde se encuentra.
    pernoctaciones.columns = pernoctaciones.columns.set_levels(pernoctaciones.columns.levels[1].str.replace('Residentes en EspaÃ±a', 'Residentes en Espana'), level=1)
    
    #A continuación junto el primer nivel y el segundo para poder exportarlo correctamente a la Base de Datos:
    pernoctaciones.columns = [e[0]+'_'+e[1]+ '_'for e in pernoctaciones.columns]    

    return pernoctaciones

# AEMET

In [5]:
def obtener_encabezado_api(api_file_path='../api.txt'):
# utilizo un token de autorización guardado en un archivo para acceder a la API de manera segura y obtener datos climáticos actualizados
    aemet = os.getenv("api")

    with open(api_file_path, 'r') as file: 
    
        token = file.read()

    header = {'Authorization': f"password {token}"}
    
    return header

In [6]:
def solicitud_get(url, api_file_path='../api.txt'):
    
    header = obtener_encabezado_api(api_file_path)

    querystring = {"api_key": header['Authorization'].split()[-1]}

    headers = {
        'cache-control': "no-cache"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        return response.json()
    else:
        # Si la solicitud no fue exitosa, imprimir el código de estado y texto de la respuesta
        print(f"Error en la solicitud. Código de estado: {response.status_code}, Mensaje: {response.text}")
        return None

In [7]:
def procesar_datos_mes(url, columnas_interes):
    # Obtener los datos de la API
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta JSON a un DataFrame
        datos = response.json()
        df = pd.DataFrame(datos)
        # Seleccionar columnas de interés
        df = df[columnas_interes]
        
        return df

In [8]:
#limpieza de datos para cada mes

def procesar_mes(df_mes):
    # Reemplazar comas por puntos y convertir las columnas numéricas a tipo float
    df_mes['tmed'] = df_mes['tmed'].str.replace(',', '.').astype(float)
    df_mes['prec'] = df_mes['prec'].replace('Acum', np.nan)
    df_mes['prec'] = df_mes['prec'].str.replace(',', '.').replace('Ip', np.nan).astype(float)
    df_mes['sol'] = df_mes['sol'].str.replace(',', '.').astype(float)
    df_mes['tmin'] = df_mes['tmin'].str.replace(',', '.').astype(float)
    df_mes['tmax'] = df_mes['tmax'].str.replace(',', '.').astype(float)
    
    # Convertir la columna "fecha" en datetime
    df_mes['fecha'] = pd.to_datetime(df_mes['fecha'])
    
    # Agrupar por provincia y calcular la media de cada columna
    df_media_mes = df_mes.groupby('provincia').agg({
        'tmed': 'mean',
        'prec': 'mean',
        'sol': 'mean',
        'tmin': 'mean',
        'tmax': 'mean'
    }).reset_index()
    
    # Renombrar las columnas para mayor claridad
    df_media_mes = df_media_mes.rename(columns={
        'tmed': 'media_tmed',
        'prec': 'media_prec',
        'sol': 'media_sol',
        'tmin': 'media_tmin',
        'tmax': 'media_tmax'
    })
    # Eliminar las filas correspondientes a 'CEUTA' y 'MELILLA'
    indices_a_eliminar = df_media_mes[df_media_mes['provincia'].isin(['CEUTA', 'MELILLA'])].index
    df_media_mes = df_media_mes.drop(indices_a_eliminar).reset_index(drop=True)
    
    return df_media_mes

In [14]:
#función para añadir aña y mes en los DF

def procesar_y_setear_mes(df, año, mes):
    df = procesar_mes(df)
    df['año'] = [año] * len(df)
    df['mes'] = [mes] * len(df)
    df.set_index('provincia', inplace=True)
    df.reset_index(inplace=True)
    return df

# CONEXIÓN SQL

In [9]:
def crear_cadena_conexion(usuario, ruta_archivo_contraseña, host, puerto, nombre_bd):

    with open(ruta_archivo_contraseña, 'r') as file:
        contraseña = file.read().strip()

    # Cadena de conexión
    cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{nombre_bd}'

    return cadena_conexion


In [10]:
def cargar_csv_a_mysql(ruta_csv, cadena_conexion, nombre_tabla):

    try:
        # Cargar el CSV en un DataFrame
        df = pd.read_csv(ruta_csv)
        
        # Crear el motor de conexión
        engine = create_engine(cadena_conexion)
        
        # Cargar el DataFrame en la base de datos
        df.to_sql(nombre_tabla, con=engine, if_exists='replace', index=False)

        print(f"Datos cargados en la tabla {nombre_tabla} con éxito.")

    except Exception as e:
        print(f"Error al cargar los datos en MySQL: {e}")